In [1]:
from __future__ import print_function

import matplotlib.pyplot as plt

import numpy as np

import os

import sys

import tarfile

import glob

from IPython.display import display, Image

from scipy import ndimage

from sklearn.linear_model import LogisticRegression

from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle

In [2]:
url = 'https://commondatastorage.googleapis.com/books1000/'
last_percent_reported = None
data_root = '/home/big_muscle/data' # Change me to store data elsewhere

In [3]:
def download_progress_hook(count, blockSize, totalSize):
  """A hook to report the progress of a download. This is mostly intended for users with
  slow internet connections. Reports every 5% change in download progress.
  """
  global last_percent_reported
  percent = int(count * blockSize * 100 / totalSize)

  if last_percent_reported != percent:
    if percent % 5 == 0:
      sys.stdout.write("%s%%" % percent)
      sys.stdout.flush()
    else:
      sys.stdout.write(".")
      sys.stdout.flush()
      
    last_percent_reported = percent

In [4]:
def maybe_download(filename, expected_bytes, force=False):
  """Download a file if not present, and make sure it's the right size."""
  dest_filename = os.path.join(data_root, filename)
  if force or not os.path.exists(dest_filename):
    print('Attempting to download:', filename) 
    filename, _ = urlretrieve(url + filename, dest_filename, reporthook=download_progress_hook)
    print('\nDownload Complete!')
  statinfo = os.stat(dest_filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified', dest_filename)
  else:
    raise Exception(
      'Failed to verify ' + dest_filename + '. Can you get to it with a browser?')
  return dest_filename

In [5]:
train_filename = maybe_download('notMNIST_large.tar.gz', 247336696)
test_filename = maybe_download('notMNIST_small.tar.gz', 8458043)

Found and verified /home/big_muscle/data/notMNIST_large.tar.gz
Found and verified /home/big_muscle/data/notMNIST_small.tar.gz


In [6]:
num_classes = 10
np.random.seed(133)

In [7]:
def maybe_extract(filename, force=False):
  root = os.path.splitext(os.path.splitext(filename)[0])[0]  # remove .tar.gz
  if os.path.isdir(root) and not force:
    # You may override by setting force=True.
    print('%s already present - Skipping extraction of %s.' % (root, filename))
  else:
    print('Extracting data for %s. This may take a while. Please wait.' % root)
    tar = tarfile.open(filename)
    sys.stdout.flush()
    tar.extractall(data_root)
    tar.close()
  data_folders = [
    os.path.join(root, d) for d in sorted(os.listdir(root))
    if os.path.isdir(os.path.join(root, d))]
  if len(data_folders) != num_classes:
    raise Exception(
      'Expected %d folders, one per class. Found %d instead.' % (
        num_classes, len(data_folders)))
  print(data_folders)
  return data_folders
  
train_folders = maybe_extract(train_filename)
test_folders = maybe_extract(test_filename)

/home/big_muscle/data/notMNIST_large already present - Skipping extraction of /home/big_muscle/data/notMNIST_large.tar.gz.
['/home/big_muscle/data/notMNIST_large/A', '/home/big_muscle/data/notMNIST_large/B', '/home/big_muscle/data/notMNIST_large/C', '/home/big_muscle/data/notMNIST_large/D', '/home/big_muscle/data/notMNIST_large/E', '/home/big_muscle/data/notMNIST_large/F', '/home/big_muscle/data/notMNIST_large/G', '/home/big_muscle/data/notMNIST_large/H', '/home/big_muscle/data/notMNIST_large/I', '/home/big_muscle/data/notMNIST_large/J']
/home/big_muscle/data/notMNIST_small already present - Skipping extraction of /home/big_muscle/data/notMNIST_small.tar.gz.
['/home/big_muscle/data/notMNIST_small/A', '/home/big_muscle/data/notMNIST_small/B', '/home/big_muscle/data/notMNIST_small/C', '/home/big_muscle/data/notMNIST_small/D', '/home/big_muscle/data/notMNIST_small/E', '/home/big_muscle/data/notMNIST_small/F', '/home/big_muscle/data/notMNIST_small/G', '/home/big_muscle/data/notMNIST_small/

In [8]:
image_size = 28  # Pixel width and height.
pixel_depth = 255.0  # Number of levels per pixel.

def load_letter(folder, min_num_images):
  """Load the data for a single letter label."""
  image_files = os.listdir(folder)
  dataset = np.ndarray(shape=(len(image_files), image_size, image_size),
                         dtype=np.float32) # dataset 包括每一个照片的每一个处理过后的像素值，是一个 N × Pixel × Pixel 的矩阵。
  print(folder)
  num_images = 0
  for image in image_files:
    image_file = os.path.join(folder, image)
    try:
      image_data = (ndimage.imread(image_file).astype(float) - 
                    pixel_depth / 2) / pixel_depth # 把所有的像素变成0-1之间的数字，有利于计算方便。
      if image_data.shape != (image_size, image_size):
        raise Exception('Unexpected image shape: %s' % str(image_data.shape))
      dataset[num_images, :, :] = image_data
      num_images = num_images + 1
    except IOError as e:
      print('Could not read:', image_file, ':', e, '- it\'s ok, skipping.')
    
  dataset = dataset[0:num_images, :, :] # 把所有 28 × 28 的图片提取出来， 过滤掉大小不一致的图片。 
  if num_images < min_num_images:
    raise Exception('Many fewer images than expected: %d < %d' %
                    (num_images, min_num_images))
    
  print('Full dataset tensor:', dataset.shape)
  print('Mean:', np.mean(dataset))
  print('Standard deviation:', np.std(dataset))
  return dataset
        
def maybe_pickle(data_folders, min_num_images_per_class, force=False):
  dataset_names = []
  for folder in data_folders:
    set_filename = folder + '.pickle'
    dataset_names.append(set_filename)
    if os.path.exists(set_filename) and not force:
      # You may override by setting force=True.
      print('%s already present - Skipping pickling.' % set_filename)
    else:
      print('Pickling %s.' % set_filename)
      dataset = load_letter(folder, min_num_images_per_class)
      try:
        with open(set_filename, 'wb') as f:
          pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)
      except Exception as e:
        print('Unable to save data to', set_filename, ':', e)
  
  return dataset_names

train_datasets = maybe_pickle(train_folders, 45000)
test_datasets = maybe_pickle(test_folders, 1800)

/home/big_muscle/data/notMNIST_large/A.pickle already present - Skipping pickling.
/home/big_muscle/data/notMNIST_large/B.pickle already present - Skipping pickling.
/home/big_muscle/data/notMNIST_large/C.pickle already present - Skipping pickling.
/home/big_muscle/data/notMNIST_large/D.pickle already present - Skipping pickling.
/home/big_muscle/data/notMNIST_large/E.pickle already present - Skipping pickling.
/home/big_muscle/data/notMNIST_large/F.pickle already present - Skipping pickling.
/home/big_muscle/data/notMNIST_large/G.pickle already present - Skipping pickling.
/home/big_muscle/data/notMNIST_large/H.pickle already present - Skipping pickling.
/home/big_muscle/data/notMNIST_large/I.pickle already present - Skipping pickling.
/home/big_muscle/data/notMNIST_large/J.pickle already present - Skipping pickling.
/home/big_muscle/data/notMNIST_small/A.pickle already present - Skipping pickling.
/home/big_muscle/data/notMNIST_small/B.pickle already present - Skipping pickling.
/hom

In [9]:
%matplotlib inline

问题1：如何显示图片？

把每个图片的像素归一化变为0至1

In [10]:
Image(filename="/home/big_muscle/data/notMNIST_small/A/MDEtMDEtMDAudHRm.png")

In [11]:
glob?

In [12]:
for file in glob.glob('/home/big_muscle/data/notMNIST_large/A/*.png')[1:100]:
    display(Image(filename=file))

问题2：
1. 显示这些数据的标签
2. 显示这些数据

In [13]:
# What is the pickle file used for？
type(train_folders)

list

In [14]:
tran_data_A = pickle.load(open(train_datasets[0], "rb"))
type(tran_data_A)
tran_data_A.mean()

-0.12825011

In [15]:
train_folders[0]

'/home/big_muscle/data/notMNIST_large/A'

问题3：观察这些被处理过的数据

Another check: we expect the data to be balanced across classes. Verify that

In [16]:
def balanced_classes(filedir):
    for file in filedir:
        mean = pickle.load(open(file, 'rb')).mean()
        print("the %s mean value is %s"%(file, mean))
balanced_classes(train_datasets)

the /home/big_muscle/data/notMNIST_large/A.pickle mean value is -0.12825
the /home/big_muscle/data/notMNIST_large/B.pickle mean value is -0.00756305
the /home/big_muscle/data/notMNIST_large/C.pickle mean value is -0.142258
the /home/big_muscle/data/notMNIST_large/D.pickle mean value is -0.0573678
the /home/big_muscle/data/notMNIST_large/E.pickle mean value is -0.069899
the /home/big_muscle/data/notMNIST_large/F.pickle mean value is -0.125583
the /home/big_muscle/data/notMNIST_large/G.pickle mean value is -0.0945816
the /home/big_muscle/data/notMNIST_large/H.pickle mean value is -0.0685222
the /home/big_muscle/data/notMNIST_large/I.pickle mean value is 0.0307862
the /home/big_muscle/data/notMNIST_large/J.pickle mean value is -0.153358


In [17]:
# DONE: 为什么下面的打包数据可以生成 标签 和数据集？
for a, b in enumerate(train_datasets):
    print(a, b)

0 /home/big_muscle/data/notMNIST_large/A.pickle
1 /home/big_muscle/data/notMNIST_large/B.pickle
2 /home/big_muscle/data/notMNIST_large/C.pickle
3 /home/big_muscle/data/notMNIST_large/D.pickle
4 /home/big_muscle/data/notMNIST_large/E.pickle
5 /home/big_muscle/data/notMNIST_large/F.pickle
6 /home/big_muscle/data/notMNIST_large/G.pickle
7 /home/big_muscle/data/notMNIST_large/H.pickle
8 /home/big_muscle/data/notMNIST_large/I.pickle
9 /home/big_muscle/data/notMNIST_large/J.pickle


In [18]:
def make_arrays(nb_rows, img_size):
    '''
    parameters: nb_rows is the valid_size, img_size is image_size
    根据训练数据的输入制造一个训练数据矩阵。
    同时和数据量对应的标签向量。
    '''
    if nb_rows:
        dataset = np.ndarray((nb_rows, img_size, img_size), dtype=np.float32)
        labels = np.ndarray(nb_rows, dtype=np.int32)
    else:
        dataset, labels = None, None
    return dataset, labels

def merge_datasets(pickle_files, train_size, valid_size=0):
  num_classes = len(pickle_files)
  valid_dataset, valid_labels = make_arrays(valid_size, image_size) # 创建验证数据集。
  train_dataset, train_labels = make_arrays(train_size, image_size) # 创建训练数据集。
  vsize_per_class = valid_size // num_classes # // 符号用来取整。
  tsize_per_class = train_size // num_classes
    
  start_v, start_t = 0, 0
  end_v, end_t = vsize_per_class, tsize_per_class
  end_l = vsize_per_class+tsize_per_class
  for label, pickle_file in enumerate(pickle_files):       
    try:
      with open(pickle_file, 'rb') as f: # 这里打开的数据集是同一个标签下的同一类的数据。
        letter_set = pickle.load(f)
        # let's shuffle the letters to have random validation and training set
        np.random.shuffle(letter_set)
        if valid_dataset is not None:
          valid_letter = letter_set[:vsize_per_class, :, :]
          valid_dataset[start_v:end_v, :, :] = valid_letter
          valid_labels[start_v:end_v] = label # 只需要标记为一个标签即可。
          start_v += vsize_per_class 
          end_v += vsize_per_class
                    
        train_letter = letter_set[vsize_per_class:end_l, :, :]
        train_dataset[start_t:end_t, :, :] = train_letter
        train_labels[start_t:end_t] = label
        start_t += tsize_per_class
        end_t += tsize_per_class
    except Exception as e:
      print('Unable to process data from', pickle_file, ':', e)
      raise
    
  return valid_dataset, valid_labels, train_dataset, train_labels
            
            
train_size = 500000
valid_size = 20000
test_size = 10000

valid_dataset, valid_labels, train_dataset, train_labels = merge_datasets(
  train_datasets, train_size, valid_size)
_, _, test_dataset, test_labels = merge_datasets(test_datasets, test_size)

print('Training:', train_dataset.shape, train_labels.shape)
print('Validation:', valid_dataset.shape, valid_labels.shape)
print('Testing:', test_dataset.shape, test_labels.shape)

Training: (500000, 28, 28) (500000,)
Validation: (20000, 28, 28) (20000,)
Testing: (10000, 28, 28) (10000,)


In [19]:
# DONE: 总共有多少数据？
def calculate_pickles(pickles):
    total = 0
    for p in pickles:
        with open(p, 'rb') as f:
            total += pickle.load(f).shape[0]
    return total

calculate_pickles(train_datasets)

529114

In [20]:
def randomize(dataset, labels):
    permutation = np.random.permutation(labels.shape[0])
    shuffled_dataset = dataset[permutation,:,:]
    shuffled_labels = labels[permutation]
    return shuffled_dataset, shuffled_labels
train_dataset, train_labels = randomize(train_dataset, train_labels)
test_dataset, test_labels = randomize(test_dataset, test_labels)
valid_dataset, valid_labels = randomize(valid_dataset, valid_labels)

In [21]:
# DONE permutation的作用，用法？
np.random.permutation(50)

array([23, 10, 25,  6, 26, 46,  9, 39,  0, 15, 20, 41,  4, 27, 19, 17, 40,
       21, 44, 22, 47, 14, 18, 43,  7, 29, 28,  8,  1, 16, 31, 35, 11, 32,
       42, 13,  2,  5, 12,  3, 34, 24, 36, 33, 38, 30, 49, 37, 48, 45])

In [22]:
pickle_file = os.path.join(data_root, 'notMNIST.pickle')

try:
  f = open(pickle_file, 'wb')
  save = {
    'train_dataset': train_dataset,
    'train_labels': train_labels,
    'valid_dataset': valid_dataset,
    'valid_labels': valid_labels,
    'test_dataset': test_dataset,
    'test_labels': test_labels,
    }
  pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
  f.close()
except Exception as e:
  print('Unable to save data to', pickle_file, ':', e)
  raise

In [23]:
statinfo = os.stat(pickle_file)
print('Compressed pickle size:', statinfo.st_size)

Compressed pickle size: 1664200515


# TODO: Use logistic regreesion to train the data.

In [24]:
with open('/home/big_muscle/data/notMNIST.pickle', 'rb') as f:
    train_dataset = pickle.load(f)

In [25]:
train_dataset.keys()

dict_keys(['train_dataset', 'train_labels', 'valid_dataset', 'valid_labels', 'test_dataset', 'test_labels'])

In [26]:
X = train_dataset['train_dataset']

In [27]:
X.shape

(500000, 28, 28)

In [28]:
X = X.reshape((500000, 28*28))

对于用来训练的矩阵，应该是一个 N × （Pixel × Pixel）的二维矩阵

In [29]:
y =train_dataset['train_labels']

In [30]:
clf = LogisticRegression(solver='lbfgs', multi_class='multinomial').fit(X, y)

In [31]:
test_dataset.shape

(10000, 28, 28)

In [32]:
clf.score(test_dataset.reshape((10000, 28*28)), test_labels)

0.89939999999999998

# Problem 6
Let's get an idea of what an off-the-shelf classifier can give you on this data. It's always good to check that there is something to learn, and that it's a problem that is not so trivial that a canned solution solves it.
Train a simple model on this data using 50, 100, 1000 and 5000 training samples. Hint: you can use the LogisticRegression model from sklearn.linear_model.
Optional question: train an off-the-shelf model on all the data!